In [ ]:
import numpy as np
import cv2
from skimage.feature import hog
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Step 1: Generate random example data (replace with your actual data loading)
# Generate random images and labels for demonstration purposes
num_samples = 100
image_height, image_width, num_channels = 224, 224, 3
X = np.random.rand(num_samples, image_height, image_width, num_channels) * 255  # Random images
y = np.random.randint(0, 2, size=num_samples)  # Random labels (0 or 1)

# Step 2: Extract features using VGG-19
def extract_vgg_features(images):
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)
    features = model.predict(images)
    return features

vgg_features = extract_vgg_features(X)

# Step 3: Extract features using HOG
def extract_hog_features(images):
    hog_features = []
    for image in images:
        fd, hog_image = hog(cv2.cvtColor(image.astype('uint8'), cv2.COLOR_RGB2GRAY), orientations=8, pixels_per_cell=(16, 16),
                            cells_per_block=(1, 1), visualize=True)
        hog_features.append(fd)
    return np.array(hog_features)

# Flatten VGG features for concatenation
vgg_features_flat = vgg_features.reshape(vgg_features.shape[0], -1)

hog_features = extract_hog_features(X)

# Step 4: Combine the features
def combine_features(vgg_features, hog_features):
    combined_features = np.concatenate((vgg_features, hog_features), axis=1)
    return combined_features

combined_features = combine_features(vgg_features_flat, hog_features)

# Step 5: Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)

# Step 6: Define a simple CNN for classification
input_layer = Input(shape=combined_features.shape[1])
x = Dense(128, activation='relu')(input_layer)
output_layer = Dense(1, activation='sigmoid')(x)  # Binary classification, so use 'sigmoid' activation

model = Model(inputs=input_layer, outputs=output_layer)

# Step 7: Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Step 8: Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Step 9: Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc*100:.2f}%')
